## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-25-15-51-43 +0000,nypost,Democrats warn Trump greenlighting Nvidia AI c...,https://nypost.com/2025/12/25/us-news/democrat...
1,2025-12-25-15-49-37 +0000,nypost,Mystery unfolds after hundreds of shoes wash u...,https://nypost.com/2025/12/25/us-news/mystery-...
2,2025-12-25-15-31-39 +0000,startribune,Vikings-Lions preview: Can Max Brosmer lead a ...,https://www.startribune.com/vikings-lions-chri...
3,2025-12-25-15-23-19 +0000,nypost,Trump admin sued by 19 states over declaration...,https://nypost.com/2025/12/25/us-news/nearly-2...
4,2025-12-25-15-22-09 +0000,nypost,Bari Weiss defends pulling ‘60 Minutes’ Salvad...,https://nypost.com/2025/12/25/media/bari-weiss...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2370/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
27,christmas,32
2,trump,24
95,new,12
125,year,8
9,china,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
84,2025-12-25-06-33-11 +0000,nypost,Trump-Kennedy Center Christmas Eve concert can...,https://nypost.com/2025/12/25/us-news/trump-ke...,76
28,2025-12-25-12-09-11 +0000,nyt,Kennedy Center’s Christmas Eve Jazz Show Cance...,https://www.nytimes.com/2025/12/24/us/kennedy-...,75
112,2025-12-25-00-41-15 +0000,bbc,Where is Santa now? Trump answers kids' Christ...,https://www.bbc.com/news/videos/cqxqvd0z5gno?a...,70
19,2025-12-25-13-23-29 +0000,nypost,Tim Walz’s daughter Hope wishes merry Christma...,https://nypost.com/2025/12/25/us-news/tim-walz...,67
8,2025-12-25-15-07-19 +0000,nypost,Trump wishes all – including ‘Radical Left Scu...,https://nypost.com/2025/12/25/us-news/trump-wi...,65


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
84,76,2025-12-25-06-33-11 +0000,nypost,Trump-Kennedy Center Christmas Eve concert can...,https://nypost.com/2025/12/25/us-news/trump-ke...
164,47,2025-12-24-19-07-18 +0000,nypost,Italian leader Giorgia Meloni gives year-end a...,https://nypost.com/2025/12/24/world-news/itali...
161,40,2025-12-24-19-26-29 +0000,nypost,DOJ unearths 1 million more possible Jeffrey E...,https://nypost.com/2025/12/24/us-news/doj-unea...
77,33,2025-12-25-08-24-14 +0000,nypost,"Powerful holiday storm lashes California, brin...",https://nypost.com/2025/12/25/us-news/southern...
127,31,2025-12-24-23-12-53 +0000,wapo,Trump-backed conservative candidate wins Hondu...,https://www.washingtonpost.com/world/2025/12/2...
0,30,2025-12-25-15-51-43 +0000,nypost,Democrats warn Trump greenlighting Nvidia AI c...,https://nypost.com/2025/12/25/us-news/democrat...
80,30,2025-12-25-07-24-27 +0000,nypost,Final photo of Arizona man killed during routi...,https://nypost.com/2025/12/25/us-news/final-ph...
152,29,2025-12-24-20-32-57 +0000,nypost,Pope Leo XIV calls for day of worldwide peace ...,https://nypost.com/2025/12/24/world-news/pope-...
4,24,2025-12-25-15-22-09 +0000,nypost,Bari Weiss defends pulling ‘60 Minutes’ Salvad...,https://nypost.com/2025/12/25/media/bari-weiss...
56,24,2025-12-25-10-30-00 +0000,wsj,Big corporations are booming on AI and record ...,https://www.wsj.com/business/the-economic-divi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
